In [1]:
import numpy as np
import time
import pandas as pd
import scipy
from joblib import load
from sklearn.gaussian_process import GaussianProcessRegressor
import sklearn.gaussian_process.kernels as k
from optim import optim
from FitGPR import fitGPR, cv_alpha_GPR
from GreeksGPR import DeltaGPR, RhoGPR
from FitGreeksGPR import fitGreeksGPR, cv_alpha_GreeksGPR
from TestPerformance import test_performanceGPR

In [2]:
# cross validation alpha for delta model
min_ = -12
max_ = -6
alphas = np.logspace(min_, max_, max_ - min_ + 1)

greeks_dataset = "Greeks1000.csv"
price_dataset = "HestonGreeks1000.csv"
validation = "HestonValidation.csv"


cv = cv_alpha_GreeksGPR().fit(
    class_=DeltaGPR,
    alphas=alphas,
    greeks_dataset=greeks_dataset,
    price_dataset=price_dataset,
    validation_dataset=validation,
)

print(f"Best Alpha:  {alphas[np.argmin(cv.aae)]}")

alpha:   1e-12
HestonValidation.csv
Time:  0.0609288215637207
AAE:  1.03e-03
MAE:  1.75e-02


alpha:   1e-11
HestonValidation.csv
Time:  0.07009243965148926
AAE:  2.12e-03
MAE:  3.01e-02


alpha:   1e-10
HestonValidation.csv
Time:  0.06192898750305176
AAE:  1.03e-03
MAE:  1.75e-02


alpha:   1e-09
HestonValidation.csv
Time:  0.07875871658325195
AAE:  1.03e-03
MAE:  1.75e-02


alpha:   1e-08
HestonValidation.csv
Time:  0.0697171688079834
AAE:  1.03e-03
MAE:  1.76e-02


alpha:   1e-07
HestonValidation.csv
Time:  0.07126808166503906
AAE:  1.04e-03
MAE:  1.78e-02


alpha:   1e-06
HestonValidation.csv
Time:  0.07207918167114258
AAE:  1.06e-03
MAE:  1.69e-02


Best Alpha:  1e-12


In [3]:
# performance of the best model based on the validation test
datasets = ["HestonTestS.csv", "HestonTestM.csv", "HestonTestL.csv"]
for dataset in datasets:
    print(dataset)
    df = pd.read_csv(dataset).to_numpy()[:, 1:]
    X = df[:, 1:]
    y = df[:, 0]
    test_performanceGPR(X, y, cv.bestAAE, type_="both", to_return=False)
    print("\n")

HestonTestS.csv
Time:  0.19776630401611328
AAE:  1.08e-03
MAE:  2.13e-02


HestonTestM.csv
Time:  0.2637941837310791
AAE:  1.03e-03
MAE:  1.63e-02


HestonTestL.csv
Time:  0.44171595573425293
AAE:  1.08e-03
MAE:  3.43e-02




In [4]:
# cross validation alpha for rho
from FitGPR import cv_alpha_GPR

min_ = -12
max_ = -6
alphas = np.logspace(min_, max_, max_ - min_ + 1)

greeks_dataset = "Greeks1000.csv"
price_dataset = "HestonGreeks1000.csv"
validation = "HestonValidation.csv"


cv = cv_alpha_GreeksGPR().fit(
    class_=RhoGPR,
    alphas=alphas,
    greeks_dataset=greeks_dataset,
    price_dataset=price_dataset,
    validation_dataset=validation,
)

print(f"Best Alpha:  {alphas[np.argmin(cv.aae)]}")

alpha:   1e-12
HestonValidation.csv
Time:  0.06668734550476074
AAE:  1.28e-03
MAE:  2.05e-02


alpha:   1e-11
HestonValidation.csv
Time:  0.07161879539489746
AAE:  1.28e-03
MAE:  2.05e-02


alpha:   1e-10
HestonValidation.csv
Time:  0.08169269561767578
AAE:  1.28e-03
MAE:  2.05e-02


alpha:   1e-09
HestonValidation.csv
Time:  0.07855987548828125
AAE:  1.27e-03
MAE:  2.04e-02


alpha:   1e-08
HestonValidation.csv
Time:  0.07182526588439941
AAE:  1.27e-03
MAE:  2.00e-02


alpha:   1e-07
HestonValidation.csv
Time:  0.06980776786804199
AAE:  1.29e-03
MAE:  1.93e-02


alpha:   1e-06
HestonValidation.csv
Time:  0.07600760459899902
AAE:  1.27e-03
MAE:  1.87e-02


Best Alpha:  1e-06


In [5]:
# performance of the best mpdel based on the validation set
datasets = ["HestonTestS.csv", "HestonTestM.csv", "HestonTestL.csv"]
for dataset in datasets:
    print(dataset)
    df = pd.read_csv(dataset).to_numpy()[:, 1:]
    X = df[:, 1:]
    y = df[:, 0]
    test_performanceGPR(X, y, cv.bestAAE, type_="both", to_return=False)
    print("\n")

HestonTestS.csv
Time:  0.20988249778747559
AAE:  1.28e-03
MAE:  2.02e-02


HestonTestM.csv
Time:  0.28121256828308105
AAE:  1.24e-03
MAE:  2.81e-02


HestonTestL.csv
Time:  0.43970632553100586
AAE:  1.27e-03
MAE:  2.45e-02




In [6]:
# let's compare it now with a standard GPR trained on the same data
# name of the dataset
name_dataset = "HestonGreeks1000.csv"

# defining the model to fit
kernel = k.RBF()
mod = GaussianProcessRegressor(
    kernel=kernel, optimizer=optim, alpha=1e-12, random_state=10
)

cv = cv_alpha_GPR().fit(
    alphas, price_dataset=name_dataset, validation_dataset=validation
)


print(f"Best Alpha:  {alphas[np.argmin(cv.aae)]}")

Fitting Time:  1.496849536895752
alpha:   1e-12
HestonValidation.csv
Time:  0.019769906997680664
AAE:  1.32e-03
MAE:  2.07e-02


Fitting Time:  1.4507577419281006
alpha:   1e-11
HestonValidation.csv
Time:  0.031122446060180664
AAE:  1.32e-03
MAE:  2.07e-02


Fitting Time:  1.7997584342956543
alpha:   1e-10
HestonValidation.csv
Time:  0.03118133544921875
AAE:  1.32e-03
MAE:  2.07e-02


Fitting Time:  1.6198270320892334
alpha:   1e-09
HestonValidation.csv
Time:  0.031215906143188477
AAE:  1.32e-03
MAE:  2.06e-02


Fitting Time:  1.6801888942718506
alpha:   1e-08
HestonValidation.csv
Time:  0.03247642517089844
AAE:  1.30e-03
MAE:  2.01e-02


Fitting Time:  1.6014032363891602
alpha:   1e-07
HestonValidation.csv
Time:  0.028772592544555664
AAE:  1.33e-03
MAE:  1.98e-02


Fitting Time:  1.4627611637115479
alpha:   1e-06
HestonValidation.csv
Time:  0.022792577743530273
AAE:  1.30e-03
MAE:  1.98e-02


Best Alpha:  1e-06


In [7]:
# performance of the best mpdel based on the validation set
datasets = ["HestonTestS.csv", "HestonTestM.csv", "HestonTestL.csv"]
for dataset in datasets:
    print(dataset)
    df = pd.read_csv(dataset).to_numpy()[:, 1:]
    X = df[:, 1:]
    y = df[:, 0]
    test_performanceGPR(X, y, cv.bestAAE, type_="both", to_return=False)
    print("\n")

HestonTestS.csv
Time:  0.12636876106262207
AAE:  1.33e-03
MAE:  2.00e-02


HestonTestM.csv
Time:  0.19022536277770996
AAE:  1.28e-03
MAE:  2.95e-02


HestonTestL.csv
Time:  0.34844136238098145
AAE:  1.32e-03
MAE:  2.58e-02


